In [1]:
import pandas as pd
import requests
import xml.etree.ElementTree as ET
import xmltodict
from datetime import datetime, timedelta
import census

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 50)

In [5]:
###########################################################
#################### RIDERSHIP ############################
###########################################################

# Source: http://64.111.127.166/origin-destination/

ridership_frame = pd.read_csv("date-hour-soo-dest-2011.csv")
ridership_frame.columns = ['day','hour','enter_abbr','exit_abbr','trip_count'] 
ridership_frame['hour'] = ridership_frame['hour'].apply(lambda x: str(x))
ridership_frame['datetime'] = ridership_frame['day'] + ' ' + ridership_frame['hour']
ridership_frame['datetime'] = ridership_frame['datetime'].apply(lambda x: datetime.strptime(x, "%Y-%m-%d %H"))
ridership_frame['day'] = ridership_frame['day'].apply(lambda x: datetime.strptime(x, "%Y-%m-%d").date())

In [6]:
ridership_frame.head(2)

,day,hour,enter_abbr,exit_abbr,trip_count,datetime
0,2011-01-01,0,12TH,16TH,1,2011-01-01
1,2011-01-01,0,12TH,24TH,3,2011-01-01


In [10]:
###########################################################
#################### STATIONS ############################
###########################################################

# Source: http://api.bart.gov/docs/stn/stns.aspx

def get_bart_stations():
    
    api_key = "ZG4U-5XSU-9XLT-DWE9"
    base_url = "http://api.bart.gov/api/stn.aspx"
    params = {"cmd" : "stns",
             "key" : api_key}

    r = requests.get(base_url, params = params)
    stations = xmltodict.parse(r.content)

    stations = stations['root']['stations'].values()[0]
    
    stations_series_list = []

    for station in stations:
        stations_series_list.append(pd.Series(station))

    station_frame = pd.concat(stations_series_list, axis=1).transpose()
    
    station_frame.columns = ['name','abbr','latitude','longitude','address','city','county','state','zipcode']
    
    return station_frame

station_frame = get_bart_stations()

In [11]:
station_frame.head(2)

,name,abbr,latitude,longitude,address,city,county,state,zipcode
0,12th St. Oakland City Center,12TH,37.803664,-122.271604,1245 Broadway,Oakland,alameda,CA,94612
1,16th St. Mission,16TH,37.765062,-122.419694,2000 Mission Street,San Francisco,sanfrancisco,CA,94110


In [ ]:
###########################################################
#################### WEATHER #############################
###########################################################

# Source: https://darksky.net/dev/docs/time-machine

def dark_sky_fetcher(station_frame, ridership_frame):
    
    # Set up
    result_list = []
    base_url = "https://api.darksky.net/forecast/"
    api_key = "54f2254db4fd0d9e7a163a2a9c38883d"
    days = ridership_frame['day'].unique()
    station_frame.index = station_frame["abbr"]

    for station in station_frame.index:
        
        query_latitude = station_frame["latitude"].loc[station]
        query_longitude = station_frame["longitude"].loc[station]
        
        for day in days:
            
            # Get midnight on selected day
            native_datetime = datetime.combine(day,datetime.min.time())
            
            # Format for Dark Sky
            query_datetime = str(native_datetime).replace(" ","T") # we omit time zone - defaults to local time
            
            # Hit the dark sky API - returns weather from all 24 hours of the day
            final_url = base_url + api_key + "/" + query_latitude + "," + query_longitude + "," + query_datetime
            r = requests.get(final_url)
            result = r.json()["hourly"]["data"]
            
            # Organize weather
            result_frame = pd.DataFrame(result)
            
            # Add station name and coordinates
            result_frame.insert(0,"station", station)
            result_frame.insert(0,"latitude", query_latitude)
            result_frame.insert(0,"longitude", query_longitude)
            
            # Log results
            result_list.append(result_frame)
            
    # Combine all results and re-order column
    weather_frame = pd.concat(result_list, axis=0)
    key_cols = ['station', "latitude", "longitude",'time']
    reordered_cols = key_cols + [col for col in weather_frame.columns if col not in key_cols]
    weather_frame = weather_frame[reordered_cols]  
    
    return weather_frame

weather_frame = dark_sky_fetcher(station_frame, ridership_frame)

# Translate time back to PT USA
weather_frame['time'] = weather_frame['time'].apply(lambda x: datetime.utcfromtimestamp(x) - timedelta(hours=8))
weather_frame.reset_index(inplace=True, drop=True)

In [24]:
weather_frame.head(2)

,station,latitude,longitude,time,apparentTemperature,cloudCover,dewPoint,humidity,icon,precipIntensity,precipProbability,precipType,pressure,summary,temperature,visibility,windBearing,windSpeed
0,12TH,37.803664,-122.271604,2011-01-01 00:00:00,45.14,1.0,38.50,0.68,cloudy,0.00,0.00,NaN,1016.67,Overcast,48.57,10.0,124.0,7.75
1,12TH,37.803664,-122.271604,2011-01-01 01:00:00,44.54,1.0,40.67,0.74,rain,0.01,0.48,rain,1016.18,Light Rain,48.57,10.0,126.0,9.25


In [2]:
###########################################################
#################### DEMOGRAPHIC ############################
###########################################################

# Source: https://www.census.gov/programs-surveys/acs/

api_key = "3bce9957462d09a7749f11461a5071c3a5afcef6"

fields_dict = {'B19083_001E' : 'gini',
'B19113_001E' : 'median_household_income',
'B19301_001E' : 'per_capita_income',
'B06009_002E' : 'some_hs',
'B06009_003E' : 'hs_grad',
'B06009_004E' : 'some_college',
'B06009_005E' : 'bach_degree',
'B06009_006E' : 'grad_degree',
'B06012_002E' : 'pov_below_100',
'B06012_003E' : 'pov_100_150',
'B06012_004E' : 'pov_150_plus',
'B19057_003E' : 'no_public_assistance',
 'B19057_002E' : 'yes_public_assistance',
'B01003_001E' : 'totalpop'}

fields = fields_dict.keys()

zip_frame = pd.read_csv("population.csv")
zips = zip_frame['zipcode'].tolist()

def get_census_metrics(year):
    
    result_series_list = []
    
    for zipcode in zips:
        
        c = census.Census(api_key, year=year)
        result = c.acs5.zipcode(fields, zipcode)
        result_series = pd.Series(result[0])
        result_series_list.append(result_series)
    
    # Organize results
    final = pd.concat(result_series_list, axis=1)
    final.columns = final.loc["zip code tabulation area"].tolist()
    final = final.reindex(final.index.drop('zip code tabulation area'))
    final['metric'] = final.index
    final['metric'].replace(fields_dict, inplace=True)
    final.drop('metric', axis=1,inplace=True)
    
    return final
        
census_frame = get_census_metrics(2011)
census_frame = census_frame.transpose()

In [3]:
census_frame.head(2)

,totalpop,some_hs,hs_grad,some_college,bach_degree,grad_degree,pov_below_100,pov_100_150,pov_150_plus,yes_public_assistance,no_public_assistance,gini,median_household_income,per_capita_income
94014,47087,5886,7614,9210,7516,1724,3979,3653,39296,452,13238,0.3822,75754,25707


In [ ]:
###########################################################
#################### REAL ESTATE ############################
###########################################################

# Source: http://www.zillow.com/research/data/

zillow_frame = pd.read_csv("Zip_Zhvi_AllHomes.csv")

# Narrow and tidy data
zillow_frame = zillow_frame[zillow_frame['RegionName'].isin(zips)]
zillow_frame.reset_index(inplace=True, drop=True)
zillow_frame.rename(columns={'RegionName' : 'zipcode'}, inplace=True)

In [ ]:
zillow_frame.head(2)